In [0]:
%fs
mkdirs dbfs:/Volumes/gizmobox/landing/operational_data/customers_sream

In [0]:

from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, TimestampType

customers_schema = StructType(fields=[StructField("customer_id", IntegerType()),
                                     StructField("customer_name", StringType()),
                                     StructField("date_of_birth", DateType()),
                                     StructField("telephone", StringType()),
                                     StructField("email", StringType()),
                                     StructField("member_since", DateType()),
                                     StructField("created_timestamp", TimestampType())
                                    ]
                              )

In [0]:

customers_df = (spark.readStream
                         .format("json")
                         .schema(customers_schema)
                         .load("/Volumes/gizmobox/landing/operational_data/customers_sream/"))

 
                         

In [0]:
from pyspark.sql.functions import current_timestamp, col
customers_transformed_df = (
                                customers_df.withColumn("file_path", col("_metadata.file_path"))
                                            .withColumn("ingestion_date", current_timestamp()))


In [0]:

streaming_query = (
                    customers_transformed_df.writeStream
                        .format("delta")
                        .trigger(processingTime='1 minutes')
                        .option("checkpointLocation", "/Volumes/gizmobox/landing/operational_data/customers_stream/_checkpoint_stream")
                        .toTable("gizmobox.bronze.customers_stream")
)

In [0]:
%sql
select * from gizmobox.bronze.customers_stream

In [0]:
%sql
select * from gizmobox.bronze.customers_stream